In [1]:
import simpy
import random

In [2]:
def cliente(env, nombre, servidores, tasa_servicio):
    """Proceso para un cliente que entra en la cola."""
    llegada = env.now
    print(f"{nombre} llegó a las {llegada:.2f}")
    
    with servidores.request() as req:
        yield req  # Espera un servidor disponible
        espera = env.now - llegada
        print(f"{nombre} es atendido tras esperar {espera:.4f} unidades de tiempo")
        
        # Tiempo de servicio (distribución exponencial)
        tiempo_servicio = random.expovariate(tasa_servicio)
        yield env.timeout(tiempo_servicio)
        print(f"{nombre} sale de la cola a las {env.now:.2f}")



In [3]:
def generar_clientes(env, tasa_llegada, tasa_servicio, servidores):
    """Genera clientes con tiempos de llegada exponenciales."""
    i = 0
    while True:
        yield env.timeout(random.expovariate(tasa_llegada))
        i += 1
        env.process(cliente(env, f"Cliente-{i}", servidores, tasa_servicio))

In [8]:
# Parámetros de la simulación
tasa_llegada = 10.0  # Tasa de llegada (media de 1 llegada por unidad de tiempo)
tasa_servicio = 1.2  # Tasa de servicio (media de 1.2 clientes atendidos por unidad de tiempo)
n_servidores = 3     # Número de servidores en paralelo
tiempo_simulacion = 10  # Tiempo total de simulación

# Crear el entorno de simulación
env = simpy.Environment()
servidores = simpy.Resource(env, capacity=n_servidores)

# Iniciar el proceso de generación de clientes
env.process(generar_clientes(env, tasa_llegada, tasa_servicio, servidores))

# Ejecutar la simulación
env.run(until=tiempo_simulacion)

Cliente-1 llegó a las 0.21
Cliente-1 es atendido tras esperar 0.0000 unidades de tiempo
Cliente-2 llegó a las 0.27
Cliente-2 es atendido tras esperar 0.0000 unidades de tiempo
Cliente-3 llegó a las 0.32
Cliente-3 es atendido tras esperar 0.0000 unidades de tiempo
Cliente-4 llegó a las 0.44
Cliente-5 llegó a las 0.46
Cliente-6 llegó a las 0.49
Cliente-7 llegó a las 0.63
Cliente-8 llegó a las 0.97
Cliente-3 sale de la cola a las 1.06
Cliente-4 es atendido tras esperar 0.6124 unidades de tiempo
Cliente-9 llegó a las 1.29
Cliente-10 llegó a las 1.39
Cliente-11 llegó a las 1.43
Cliente-12 llegó a las 1.47
Cliente-13 llegó a las 1.51
Cliente-14 llegó a las 1.53
Cliente-15 llegó a las 1.53
Cliente-16 llegó a las 1.58
Cliente-17 llegó a las 1.64
Cliente-18 llegó a las 1.74
Cliente-19 llegó a las 1.76
Cliente-20 llegó a las 1.94
Cliente-21 llegó a las 1.94
Cliente-22 llegó a las 1.95
Cliente-1 sale de la cola a las 1.97
Cliente-5 es atendido tras esperar 1.5069 unidades de tiempo
Cliente-23 lle

In [7]:
import pandas as pd

In [14]:
df = pd.read_csv("resultados_simulacion.csv")
df.head()

,ID de simulación,Numero de cliente,Hora de llegada,Tiempo entre llegada,Tiempo de llegada,Tiempo en Cola,Hora de inicio de servicio,Tiempo de servicio,Hora de fin de servicio,Tiempo en el sistema,Longitud del sistema,Longitud de la Cola
0,1,Cliente-1,0:00:51,0.000000,50.559593,0.0,0:00:51,405.992194,0:07:37,405.992194,0,0
1,1,Cliente-2,0:08:12,441.149342,491.708935,0.0,0:08:12,154.428656,0:10:46,154.428656,0,0
2,1,Cliente-3,0:11:46,214.342149,706.051084,0.0,0:11:46,65.514403,0:12:52,65.514403,0,0
3,1,Cliente-5,0:14:09,42.991368,849.344082,0.0,0:14:09,89.599087,0:15:39,89.599087,1,0
4,1,Cliente-4,0:13:26,100.301629,806.352714,0.0,0:13:26,226.285430,0:17:13,226.285430,0,0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID de simulación            935 non-null    int64  
 1   Numero de cliente           935 non-null    object 
 2   Hora de llegada             935 non-null    object 
 3   Tiempo entre llegada        935 non-null    float64
 4   Tiempo de llegada           935 non-null    float64
 5   Tiempo en Cola              935 non-null    float64
 6   Hora de inicio de servicio  935 non-null    object 
 7   Tiempo de servicio          935 non-null    float64
 8   Hora de fin de servicio     935 non-null    object 
 9   Tiempo en el sistema        935 non-null    float64
 10  Longitud del sistema        935 non-null    int64  
 11  Longitud de la Cola         935 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 87.8+ KB


In [20]:
sum(df.loc[df["ID de simulación"] == 1,"Tiempo en Cola"])

4565.449847890729

In [28]:
# Agrupar por ID de simulación
agrupado_por_simulacion = df.groupby('ID de simulación')

# Calcular métricas por simulación
resultados = agrupado_por_simulacion.agg(
    longitud_esperada_sistema=('Longitud del sistema', 'mean'),
    longitud_promedio_cola=('Longitud de la Cola', 'mean'),
    tiempo_total_espera=('Tiempo de servicio', 'sum'),
    tiempo_promedio_sistema=('Tiempo en el sistema', 'mean'),
    tiempo_total_cola=('Tiempo en Cola', 'sum'),
    promedio_tiempo_cola=('Tiempo en Cola', 'mean'),
    maxima_longitud_cola=('Longitud de la Cola', 'max')
).reset_index()

In [29]:
resultados.head()

,ID de simulación,longitud_esperada_sistema,longitud_promedio_cola,tiempo_total_espera,tiempo_promedio_sistema,tiempo_total_cola,promedio_tiempo_cola,maxima_longitud_cola
0,1,0.930233,0.069767,27501.453794,372.870973,4565.449848,53.086626,1
1,2,1.239583,0.260417,30703.057572,427.293586,10317.126681,107.470070,3
2,3,0.768116,0.101449,21680.220403,356.785003,2937.944777,42.578910,2
3,4,1.152941,0.247059,25215.650173,388.829703,7834.874602,92.174995,4
4,5,0.850746,0.179104,22935.084891,423.434849,5435.050017,81.120150,3


# ¿Qué hace `SimPy` para simular el sistema de colas?

La biblioteca `SimPy` es un marco para la simulación de eventos discretos en Python. En este caso, se utiliza para modelar un sistema de colas con múltiples servidores y clientes. A continuación, se detalla cómo `SimPy` facilita la simulación:

---

## **1. Modelado del tiempo**

`SimPy` gestiona el tiempo de manera discreta, es decir, avanza el tiempo de simulación de un evento al siguiente. Esto permite simular procesos donde los eventos ocurren en momentos específicos (como llegadas de clientes o liberación de servidores).

En el código:
- `env.now`: Proporciona el tiempo actual en la simulación.
- `env.timeout(t)`: Pausa el proceso por `t` unidades de tiempo, simulando el paso del tiempo entre eventos.

---

## **2. Creación de procesos**

`SimPy` modela cada entidad (como un cliente) como un proceso independiente que realiza una serie de pasos. Estos procesos se definen como funciones generadoras en Python, donde `yield` se usa para ceder el control al entorno de simulación.

En el código:
- Cada cliente se modela como un proceso mediante la función `cliente`.
- Los procesos de clientes se crean dinámicamente con `env.process()` y se gestionan en paralelo.

---

## **3. Gestión de recursos**

`SimPy` permite modelar recursos con capacidad limitada, como servidores, máquinas o espacios de almacenamiento. Estos recursos controlan el acceso de los procesos y gestionan las colas cuando la capacidad está llena.

En el código:
- `simpy.Resource(env, capacity=n)`: Crea un recurso con una capacidad de `n` unidades (en este caso, los servidores).
- `servidores.request()`: Simula que un cliente solicita acceso a un servidor.
- `with servidores.request() as req`: Modelo de contexto para manejar la ocupación y liberación automática del recurso.

---

## **4. Simulación de distribuciones aleatorias**

`SimPy` permite integrar distribuciones aleatorias, como la exponencial, para modelar la incertidumbre en los tiempos entre llegadas o los tiempos de servicio.

En el código:
- `random.expovariate(tasa)`: Genera valores aleatorios siguiendo una distribución exponencial, útil para modelar:
  - Tiempos entre llegadas de clientes (`tasa_llegada`).
  - Duración del servicio a un cliente (`tasa_servicio`).

---

## **5. Coordinación y ejecución de la simulación**

`SimPy` se encarga de coordinar todos los procesos y eventos, avanzando el tiempo y ejecutando las acciones en el orden correcto.

En el código:
- `env.run(until=t)`: Ejecuta la simulación hasta el tiempo `t`, gestionando las interacciones entre los procesos y los recursos.

---

En resumen, `SimPy` proporciona herramientas para modelar, coordinar y ejecutar simulaciones de eventos discretos, lo que permite analizar sistemas complejos como colas y tiempos de espera de forma eficiente y reproducible.
